In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import lightning as L
from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch.loggers import TensorBoardLogger

from graphFeatureSelect.datamodule import AnnDataGraphDataModule
from graphFeatureSelect.utils import get_datetime, get_paths
import numpy as np
import random
import torch

def setup_seeds(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


seed = 42
setup_seeds(seed)
# data parameters, we'll eventually obtain this from the data. 
n_genes = 500
n_labels = 126

# paths
paths = get_paths()



In [3]:
import numpy as np
np.array([1,2,3])

array([1, 2, 3])

In [4]:

# data, model and fitting
datamodule = AnnDataGraphDataModule(data_dir=paths["data_root"], file_names=["VISp_nhood.h5ad"], batch_size=1)


In [5]:
datamodule.setup()

In [6]:
datamodule.data

Data(x=[61884, 500], edge_index=[2, 1949970], labels=[61884], train_mask=[61884], val_mask=[61884], test_mask=[61884])

In [7]:
train_loader = datamodule.train_dataloader()

/data/users1/dkim195/miniconda3/envs/allen2/lib/python3.9/site-packages/torch_geometric/sampler/neighbor_sampler.py:55: UserWarning: The usage of the 'directed' argument in 'NeighborSampler' is deprecated. Use `subgraph_type='induced'` instead.
  warnings.warn(f"The usage of the 'directed' argument in "
/data/users1/dkim195/miniconda3/envs/allen2/lib/python3.9/site-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [8]:
batch = next(iter(train_loader))

In [9]:
batch

Data(x=[267, 500], edge_index=[2, 5237], labels=[267], train_mask=[267], val_mask=[267], test_mask=[267], n_id=[267], e_id=[5237], input_id=[1], batch_size=1)

In [10]:
from graphFeatureSelect.models import GAT

In [11]:
model = GAT(hidden_channels=32, num_features = 500, num_classes=126)

In [14]:
data = batch
gene_exp = data.x
edgelist = data.edge_index
celltype = data.labels
gene_exp = gene_exp.float()


In [15]:
model(gene_exp, edgelist)

tensor([[ 0.3697,  0.7600, -0.4773,  ..., -0.2104, -0.8952, -0.2712],
        [-0.0720,  0.5445, -0.8735,  ..., -0.3121, -1.0241,  0.0154],
        [ 0.4845,  0.3228, -0.7457,  ...,  0.2163, -1.4714, -0.4191],
        ...,
        [ 0.6444,  0.4359, -0.6680,  ..., -0.2457, -0.9161,  0.0059],
        [ 1.0631,  0.1950, -0.2538,  ...,  0.3416, -0.8997, -0.0147],
        [ 0.9191, -0.1468, -0.4413,  ...,  0.1666, -0.4205,  0.0287]],
       grad_fn=<AddBackward0>)

In [ ]:
csr = datamodule.data.edge_index
coo_matrix = csr.tocoo()
values = coo_matrix.data
indices = np.vstack((coo_matrix.row, coo_matrix.col))
indices = torch.LongTensor(indices)
values = torch.FloatTensor(values)
shape = coo_matrix.shape
sparse_tensor = torch.sparse_coo_tensor(indices, values, torch.Size(shape))
edge_index = sparse_tensor._indices()


In [ ]:
edge_index

In [ ]:
datafull.celltypes()[2131]

In [ ]:
datafull[0][0]